In [56]:
#****************************************************************************************#
# Program name      : Application                                                        #
# Project           : Online Book Review Database Management System with MYSQL           #
# Description       : IDMP_Project                                                       #
# Produced by       : APOORVA GUPTA AND PRIYA GARG                                       #
# Date              : 12/11/2021                                                         #
#****************************************************************************************#

# Online Book Review DBMS using Tkinter

In [3]:
# Importing all libraries

import pandas as pd 
import numpy as np

import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter import messagebox

from tkcalendar import Calendar

from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [4]:
today_date = date.today()

In [5]:
# establishing MYSQL connection

from mysql.connector import connect, Error
from getpass import getpass

try:
    conn=connect(
        host='localhost',
        port=3306,
        user='root',
        password='rootroot',
        auth_plugin='mysql_native_password',
        database='goodreads'
        )
except Error as e:
    print(e)

### SQL Queries and Functions

In [6]:

##### SQL Queries and Functions ######



######################################____TAB1___BOOK_SEARCH_______######################################
returned_books = []

#function to search book by book title
def book_search_sql(book_title):
    
    #to fetch all books
    book_search_query="""                                 
                        select b.*
                        from books as b
                    """
    with conn.cursor() as cursor:
        cursor.execute(book_search_query)
        result=cursor.fetchall()

    new_result=[]
    
    #to check if entered book title is present in database
    for x in result:
        if book_title in x[0]:
            new_result.append(x)
    
    return new_result

        
######################################____TAB2___USER_SEARCH_______######################################

#function to search user by username
def reader_search_sql(username):
    
    reader_search_query="""
        select r.Username AS Username,r.Password AS Password,
        r.Email_id AS Email_id,r.DOB AS DOB,
        count(distinct v.Review_id) AS no_of_reviews,
        avg(v.Rating) AS avg_rating,count(distinct w.Book_id) AS wants_to_read 
        from reader as r 
        left join review v on r.Username = v.username 
        left join wants_to_read as w on r.Username = w.username
        where r.username=%s
    """
    record=(username)
    with conn.cursor() as cursor:
        cursor.execute(reader_search_query,(record,))
        result=cursor.fetchall()
    return result

    
######################################____TAB3___REVIEW_______######################################


#function to add review by inserting all attributes of review. 
    #Adding review also updates total no of reviews for that book by 1 and available no fo reviews for that book id and username by -1
def add_review_sql(review_id,timestamp,description,recommended,rating,no_of_likes,a_username,a_book_id):
    cursor = conn.cursor()
    add_review_query="""
    insert into review(review_id,timestamp,description,recommended,rating,no_of_likes,
    username,book_id)
    values (%s,%s,%s,%s,%s,%s,%s,%s)
    """
    record=(review_id,timestamp,description,recommended,rating,no_of_likes,a_username,a_book_id)
    cursor.execute(add_review_query,record)

    update_review_query="""
    update books
    set no_of_reviews = no_of_reviews+1
    where book_id = %s
    """
    data=(a_book_id,)
    cursor.execute(update_review_query,data)

    update_available_review_query="""
    update reading
    set Available_no_of_reviews = Available_no_of_reviews-1
    where book_id=%s and username=%s
    """
    data_update=(a_book_id,a_username)
    cursor.execute(update_available_review_query,data_update)

    conn.commit()
    

    
######################################____TAB4___READER_______######################################

##function to add/insert new reader
def add_reader_sql(username,password,Email_id,dob):
    cursor = conn.cursor()
    add_reader_query="""
    insert into reader(username,password,Email_id,dob)
    values (%s,%s,%s,%s)
    """
    record=(username,password,Email_id,dob)
    cursor.execute(add_reader_query,record)
    conn.commit()
    
       
##function to delete exisiting reader using username
def delete_reader_sql(username):
    cursor = conn.cursor()
    add_delete_query="""
        delete from reader where username=%s
    """
    record=(username)
    cursor.execute(add_delete_query,(record,))
    conn.commit()

    
##function to add user for book_id in reading table
def add_reader_in_reading(username):
    
    book_id = '960'
    num_available_reviews = 2
    
    cursor = conn.cursor()
    add_reading_query="""
    insert into reading(username,book_id,available_no_of_reviews)
    values (%s,%s,%s)
    """
    record=(book_id,username,num_available_reviews)
    cursor.execute(add_reading_query,record)
    conn.commit()

######################################____TAB5___BOOK_______######################################


##function to add/insert new book in database
def add_book_sql(book_title,book_id,no_of_reviews,no_of_pages,author_name,genre):
    cursor = conn.cursor()
    add_book_query="""
    insert into books(book_title,book_id,no_of_reviews,no_of_pages,author_name,genre)
    values (%s,%s,%s,%s,%s,%s)
    """
    record=(book_title,book_id,no_of_reviews,no_of_pages,author_name,genre)
    cursor.execute(add_book_query,record)
    conn.commit()
    
    
##function to delete exisiting book using book title
def delete_book_sql(book_id):
    cursor = conn.cursor()
    delete_book_publish_query="""
        delete a from Books_Publisher_name as a inner join books b on a.book_id=b.book_id
         where b.book_id=%s
    """
    delete_book_query="""
        delete from books where book_id=%s
    """
    record=(book_id)
    cursor.execute(delete_book_publish_query,(record,))
    cursor.execute(delete_book_query,(record,))
    conn.commit()

### User Sentiment For Books (extracted through Sentiment Analysis of Reviews)

In [7]:
# reading predictions data 
df = pd.read_csv('../data/all_predictions_sentiments.csv')
# df.head()

In [8]:
# extracting number of positive, negative and neutral feedbacks from users to each book
positive_sentiment = (df[df.prediction_lr == 1].groupby('book_id').prediction_lr.count()
     .reindex(df.book_id.unique()).fillna(0).astype(int)
     .rename('positive_num').reset_index())

negative_sentiment = (df[df.prediction_lr == -1].groupby('book_id').prediction_lr.count()
     .reindex(df.book_id.unique()).fillna(0).astype(int)
     .rename('negative_num').reset_index())

neutral_sentiment = (df[df.prediction_lr == 0].groupby('book_id').prediction_lr.count()
     .reindex(df.book_id.unique()).fillna(0).astype(int)
     .rename('neutral_num').reset_index())

# combined positive, negative and neutral feedback count
sentiment = pd.merge(pd.merge(positive_sentiment,negative_sentiment,on='book_id'),neutral_sentiment,on='book_id')

# sentiment.head()

In [9]:
# calculating positive and negative %
sentiment['total_sentiments'] = sentiment['positive_num']+sentiment['negative_num']+sentiment['neutral_num']
sentiment['positive_%'] = 100*sentiment['positive_num']/sentiment['total_sentiments']
sentiment['negative_%'] = 100*sentiment['negative_num']/sentiment['total_sentiments']

In [10]:
# function to get overall user sentiment for books having relevant reviews (that is, review with atleast 1 upvote)
def get_sentiment(book_id):
    book_id_for_sentiment = book_id
    selected_book = sentiment[sentiment['book_id'] == int(book_id_for_sentiment)].reset_index()

    if len(selected_book) == 0:
        temp_sentiment = 'Not Available'
    else:
        temp_sentiment = str(int(selected_book['positive_%'][0])) + '% positive and ' + str(int(selected_book['negative_%'][0])) + '% negative'        
    
    return temp_sentiment


### Tkinter functions

#### The GUI for this database has been developed with Tkinter. Please use the following command to download it.

In [11]:
# pip install tkinter

#### All functions used in the application are defined below. Some functions call previously defined SQL query functions, such as, search, insert or delete to connect to MYSQL Database.

In [12]:

######################################____TAB1___BOOK_SEARCH_______######################################

# search for a book by title
def search_book():
    
    # empty fields will raise an error
    if book_id_text_search.get() == '':
        messagebox.showerror('Required Fields', 'Please include all fields')
        return
    
    books_list.delete(0, END) 
    
    # connecting to MYSQL for book search 
    result_book = []
    result_book = book_search_sql(book_id_text_search.get())   
    
    i = 0
    
    for book in result_book:
        
        # getting user sentiment from sentiment analysis model
        temp_sentiment = get_sentiment(list(book)[1])
        
        i = i + 1
        temp_list = []
        
        local_dict = {'Search':i, 'title':list(book)[0], 'ID': list(book)[1],
                     'Author':list(book)[4],'genre':list(book)[5],
                      'num_pages':list(book)[3], 'num_reviews': list(book)[2],
                      'user_sentiment': temp_sentiment  }
        
        # showing search list in interface
        for k, v in local_dict.items():
            books_list.insert(END, [k, '-->', v])

    clear_booksearch()

# function for clearing the temporary variables created in interface 
def clear_booksearch():
    book_id_entry.delete(0, END)
    
    
######################################____TAB2___USER_SEARCH_______######################################

# search for reader by username
def search_user():
    if user_id_text_search.get() == '':
        messagebox.showerror('Required Fields', 'Please include all fields')
        return
    
    users_list.delete(0, END)
    
    # connecting to MYSQL for user search
    result_user = reader_search_sql(user_id_text_search.get())
        
    for user in result_user:
        
        temp_list = []
        local_dict = {'user':list(user)[0], 'password': list(user)[1],
                     'email_id':list(user)[2],'dob':list(user)[3]}
        
        # showing search list in interface
        for k, v in local_dict.items():
            users_list.insert(END, [k, '-->', v])
    
    clear_usersearch()
    
# function for clearing the temporary variables created in interface 
def clear_usersearch():
    user_id_entry_search.delete(0, END)
    
    
    
######################################____TAB3___REVIEW_______######################################

# adding review to database
def add_review():
    
    # empty fields will raise an error
    if book_id_reader_text.get() == '' or username_reader_text.get() == '' or rating_reader_text.get() == '' or review_reader_text.get() == '' or num_likes_reader_text.get() == '':
        messagebox.showerror('Required Fields', 'Please include all fields')
        return
    
    # rating should be between 1 and 5
    if int(rating_reader_text.get()) >= 6 or int(rating_reader_text.get()) <=0:
        messagebox.showerror('Error', 'Rating must be between 1 and 5')
        return
    
    # threshold to decide whether a user recommends the book or not
    if int(rating_reader_text.get()) >= 3 :
        recommend = 'Y'
    else:
        recommend = 'N'
    
    # review id is the primary key in review table. So adding index for every new review being added.
    query_for_max_reviewID = """
                            select max(review_id) from review
                        """
    
    with conn.cursor() as cur:

        cur.execute(query_for_max_reviewID)
        local_result = cur.fetchall()
        for row in local_result:
            max_val = row
    
    max_val = list(max_val)
    if max_val[0] == None:
        max_val[0] = 0
        
    new_review_id = max_val[0]+1    
        
    # connecting to MYSQL to insert review
    add_review_sql(new_review_id, today_date, review_reader_text.get(), recommend,int(rating_reader_text.get()),
                   int(num_likes_reader_text.get()), username_reader_text.get(), book_id_reader_text.get())
    
    clear_review()
    
# function for clearing the temporary variables created in interface     
def clear_review():
    book_id_reader_entry.delete(0, END)
    username_reader_entry.delete(0, END)
    rating_reader_entry.delete(0, END)
    review_reader_entry.delete(0, END)
    num_likes_reader_entry.delete(0, END)


######################################____TAB4___READER_______######################################

# adding user to database
def add_user():
    
    # empty fields will raise an error
    if username_text.get() == '' or password_text.get() == '' or email_text.get() == '':
        messagebox.showerror('Required Fields', 'Please include all fields')
        return
    
    # checking whether user already exists in the database or not
    user_check_query = """
                    select * from reader where Username = %s 
                """
    val_tuple = (username_text.get())
    
    with conn.cursor() as cur_user:
        cur_user.execute(user_check_query,(val_tuple,))
        result=cur_user.fetchall()
    
    if len(result) > 0:
        messagebox.showerror('Error', 'username already taken\nplease try another')
        #clear_user()
        return
    
    # connecting to MYSQL to insert user
    add_reader_sql(username_text.get(),password_text.get(),email_text.get(),cal.get_date())
    
    # add_reader_in_reading(username_text.get())
    
    clear_user()
    
# deleting user from database
def delete_user():
    delete_reader_sql(username_text.get())
    clear_user()
    
# function for clearing the temporary variables created in interface    
def clear_user():
    username_entry.delete(0, END)
    password_entry.delete(0, END)
    email_entry.delete(0, END)
    # dob_entry.delete(0, END)

# getting date for user dob
def grad_date():
    dob.config(text = "Selected Date is: " + cal.get_date())


    
######################################____TAB5___BOOK_______######################################

# inserting new book into database
def add_book():
    
    # query to check whether the new book id being inserted already exists in the database or not 
    book_check_query = """
                    select * from Books where Book_id = %s 
                """
    val_tuple = (book_id_text.get())
    
    with conn.cursor() as cur_book:
        cur_book.execute(book_check_query,(val_tuple,))
        result=cur_book.fetchall()
    
    if len(result) > 0:
        messagebox.showerror('Error', 'Book ID already exists')
        #clear_user()
        return
    
    # empty fields will raise an error
    if book_id_text.get() == '' or book_title_text.get() == '' or author_text.get() == '' or genre_text.get() == '' or num_pages_text.get() == '' or num_reviews_text.get() == '':
        messagebox.showerror('Required Fields', 'Please include all fields')
        return

    # num_pages should be positive
    if num_pages_text.get() <= 0:
        messagebox.showerror('Error', 'Number of pages should be positive')
        return
    

    # connecting to MYSQL to insert book
    add_book_sql(book_title_text.get(),book_id_text.get(),num_reviews_text.get(),
                     num_pages_text.get(),author_text.get(),genre_text.get())
    
    clear_book()
    
# deleting book from database
def delete_book():
    delete_book_sql(book_id_text.get())
    clear_book()
    
# function for clearing the temporary variables created in interface     
def clear_book():
    book_id_entry.delete(0, END)
    book_title_entry.delete(0, END)
    author_entry.delete(0, END)
    genre_entry.delete(0, END)
    num_pages_entry.delete(0, END)
    num_reviews_entry.delete(0, END)


In [13]:
# root window
app = tk.Tk()
app.geometry('600x400')
app.title('Online Book Review System')

# create a notebook
notebook = ttk.Notebook(app)
notebook.pack(pady=10, expand=True)


######################################____TAB1___BOOK_SEARCH_______######################################

# create frames
frame1 = ttk.Frame(notebook, width=400, height=280)
frame1.pack(fill='both', expand=True)

# add frames to notebook
notebook.add(frame1, text='Book Search')


# book search details
book_id_text_search = StringVar()
book_id_label_search = Label(frame1, text='Book title', font=('bold', 14), pady=20)
book_id_label_search.grid(row=0, column=0, sticky=W)
book_id_entry_search = Entry(frame1, textvariable=book_id_text_search)
book_id_entry_search.grid(row=0, column=1)

# Buttons
search_btn = Button(frame1, text='Search Book', width=12, command=search_book)
search_btn.grid(row=4, column=0, pady=20)

# creating a listbox to show the search results
books_list = Listbox(frame1, height=8, width=50, border=0)
books_list.grid(row=5, column=0, columnspan=3, rowspan=6, pady=20, padx=20)
# Create scrollbar
scrollbar = Scrollbar(frame1)
scrollbar.grid(row=5, column=3)
# Set scroll to listbox
books_list.configure(yscrollcommand=scrollbar.set)
scrollbar.configure(command=books_list.yview)


######################################____TAB2__USER_SEARCH___######################################

# frame 
frame2 = ttk.Frame(notebook, width=400, height=280)
frame2.pack(fill='both', expand=True)
notebook.add(frame2, text='User Profile Search')

# user search details
user_id_text_search = StringVar()
user_id_label_search = Label(frame2, text='username', font=('bold', 14), pady=20)
user_id_label_search.grid(row=0, column=0, sticky=W)
user_id_entry_search = Entry(frame2, textvariable=user_id_text_search)
user_id_entry_search.grid(row=0, column=1)

# Buttons
search_btn = Button(frame2, text='Search user', width=12, command=search_user)
search_btn.grid(row=4, column=0, pady=20)

# creating a listbox to show the search results
users_list = Listbox(frame2, height=8, width=50, border=0)
users_list.grid(row=5, column=0, columnspan=3, rowspan=6, pady=20, padx=20)
# Create scrollbar
scrollbar = Scrollbar(frame2)
scrollbar.grid(row=5, column=3)
# Set scroll to listbox
users_list.configure(yscrollcommand=scrollbar.set)
scrollbar.configure(command=users_list.yview)


######################################____TAB3__REVIEW___######################################

# frame
frame3 = ttk.Frame(notebook, width=400, height=280)
frame3.pack(fill='both', expand=True)
notebook.add(frame3, text='Review')


# Book ID
book_id_reader_text = StringVar()
book_id_reader_label = Label(frame3, text='Book ID', font=('bold', 14), pady=20)
book_id_reader_label.grid(row=0, column=0, sticky=W)
book_id_reader_entry = Entry(frame3, textvariable=book_id_reader_text)
book_id_reader_entry.grid(row=0, column=1)

# user name
username_reader_text = StringVar()
username_reader_label = Label(frame3, text='UserName', font=('bold', 14))
username_reader_label.grid(row=1, column=0, sticky=W)
username_reader_entry = Entry(frame3, textvariable=username_reader_text)
username_reader_entry.grid(row=1, column=1)

# Rating
rating_reader_text = StringVar()
rating_reader_label = Label(frame3, text='Rating (from 1 to 5)', font=('bold', 14))
rating_reader_label.grid(row=2, column=0, sticky=W)
rating_reader_entry = Entry(frame3, textvariable=rating_reader_text)
rating_reader_entry.grid(row=2, column=1)

# Review_text
review_reader_text = StringVar()
review_reader_label = Label(frame3, text='Review', font=('bold', 14))
review_reader_label.grid(row=2, column=2, sticky=W)
review_reader_entry = Entry(frame3, textvariable=review_reader_text)
review_reader_entry.grid(row=2, column=3)

# num_likes
num_likes_reader_text = StringVar()
num_likes_reader_label = Label(frame3, text='# likes', font=('bold', 14))
num_likes_reader_label.grid(row=3, column=2, sticky=W)
num_likes_reader_entry = Entry(frame3, textvariable=num_likes_reader_text)
num_likes_reader_entry.grid(row=3, column=3)


# Buttons
add_btn = Button(frame3, text='Add Review', width=12, command=add_review)
add_btn.grid(row=4, column=0, pady=20)

clear_btn = Button(frame3, text='Clear Input', width=12, command=clear_review)
clear_btn.grid(row=4, column=3)



######################################____TAB4__READER___######################################

# frame
frame4 = ttk.Frame(notebook, width=400, height=280)
frame4.pack(fill='both', expand=True)
notebook.add(frame4, text='Reader')


# user name
username_text = StringVar()
username_label = Label(frame4, text='UserName', font=('bold', 14))
username_label.grid(row=0, column=0, sticky=W)
username_entry = Entry(frame4, textvariable=username_text)
username_entry.grid(row=0, column=1)

# text 
user_delete_label = Label(frame4, text='Only username required for delete', font=('bold', 14))
user_delete_label.grid(row=1, column=1, sticky=W)

# Password
password_text = StringVar()
password_label = Label(frame4, text='Password', font=('bold', 14))
password_label.grid(row=0, column=2, sticky=W)
password_entry = Entry(frame4, textvariable=password_text)
password_entry.grid(row=0, column=3)

# Email_id
email_text = StringVar()
email_label = Label(frame4, text='Email ID', font=('bold', 14))
email_label.grid(row=2, column=2, sticky=W)
email_entry = Entry(frame4, textvariable=email_text)
email_entry.grid(row=2, column=3)

# DOB
dob_label = Label(frame4, text='Date of Birth', font=('bold', 14))
dob_label.grid(row=3, column=2, sticky=W)

# calender for selecting date
cal = Calendar(frame4, selectmode = 'day', year = 2021, month = 12, day = 15)
cal.grid(row=3,column=3)

# Button to get the selected date
Button(frame4, text = "Get Date", command = grad_date).grid(row=5,column=3)

# Label for the date
dob = Label(frame4, text = "")
dob.grid(row=4,column=3)


# Buttons
add_btn = Button(frame4, text='Add User', width=12, command=add_user)
add_btn.grid(row=4, column=0, pady=20)

clear_btn = Button(frame4, text='Clear Input', width=12, command=clear_user)
clear_btn.grid(row=5, column=0)

delete_btn = Button(frame4, text='Delete User', width=12, command=delete_user)
delete_btn.grid(row=6, column=0)


######################################____TAB5__BOOK___######################################

# frame
frame5 = ttk.Frame(notebook, width=400, height=280)
frame5.pack(fill='both', expand=True)
notebook.add(frame5, text='Book')

# Book ID
book_id_text = StringVar()
book_id_label = Label(frame5, text='Book ID', font=('bold', 14))
book_id_label.grid(row=0, column=0, sticky=W)
book_id_entry = Entry(frame5, textvariable=book_id_text)
book_id_entry.grid(row=0, column=1)

# text
book_id_label = Label(frame5, text='Only Book ID required for delete', font=('bold', 14))
book_id_label.grid(row=1, column=1, sticky=W)

# Book Title
book_title_text = StringVar()
book_title_label = Label(frame5, text='Book Title', font=('bold', 14))
book_title_label.grid(row=0, column=2, sticky=W)
book_title_entry = Entry(frame5, textvariable=book_title_text)
book_title_entry.grid(row=0, column=3)

# Author name
author_text = StringVar()
author_label = Label(frame5, text='Author Name', font=('bold', 14))
author_label.grid(row=2, column=0, sticky=W)
author_entry = Entry(frame5, textvariable=author_text)
author_entry.grid(row=2, column=1)

# Genre
genre_text = StringVar()
genre_label = Label(frame5, text='Genre', font=('bold', 14))
genre_label.grid(row=3, column=0, sticky=W)
genre_entry = Entry(frame5, textvariable=genre_text)
genre_entry.grid(row=3, column=1)

# number of pages
num_pages_text = IntVar()
num_pages_label = Label(frame5, text='num_pages', font=('bold', 14))
num_pages_label.grid(row=2, column=2, sticky=W)
num_pages_entry = Entry(frame5, textvariable=num_pages_text)
num_pages_entry.grid(row=2, column=3)

# num_reviews
num_reviews_text = StringVar()
num_reviews_label = Label(frame5, text='num_reviews', font=('bold', 14))
num_reviews_label.grid(row=3, column=2, sticky=W)
num_reviews_entry = Entry(frame5, textvariable=num_reviews_text)
num_reviews_entry.grid(row=3, column=3)


# Buttons
add_btn = Button(frame5, text='Add Book', width=12, command=add_book)
add_btn.grid(row=4, column=0, pady=20)

clear_btn = Button(frame5, text='Clear Input', width=12, command=clear_book)
clear_btn.grid(row=4, column=3)

delete_btn = Button(frame5, text='Delete Book', width=12, command=delete_book)
delete_btn.grid(row=5, column=0)

# to keep the app window open
app.mainloop()